### Raw Data EDA

In [1]:
# import libraries

import os
import io
import re
import csv
from string import punctuation
from time import time
from nltk.tokenize import sent_tokenize, word_tokenize

from statistics import *
import numpy as np

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/lea/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

#### Helper Functions

In [3]:
def process_ann(ann_file):
    """Helper function that reads a .ann file,
       strips out newline characters, splits the tab-delimited entries,
       and extracts information for identifying entities and relations 
       in corresponding .txt file
       
       Input:
       ann_file = tab-delimited brat annotation file with the following format
                  NER: [entity_ID]\t[label start_offset end_offset]\t[entity]
                  RE:  [relation_ID]\t[relation_type argument1 argument2]
       
       Outputs:
       cleaned_offsets = list of tuples for labeling corresponding .txt file
                         format: (offset, label, entity ID)
       relations = list of tuples for extracting relations from corresponding .txt file
                   format: (relation ID, relation_type, entity ID #1, entity ID #2)"""
    
    with io.open(ann_file, 'r', encoding='utf-8', errors='ignore') as f:
        text = [x.strip().split('\t') for x in f.readlines()]
        
    ann = [x for x in text if x[0][0] == 'T']
    rel = [x for x in text if x[0][0] == 'R']
    
    # extract information for identifying entities
    offsets = []
    
    for x in ann:
        entity_id = x[0]
        start = int(x[1].split()[1])
        end = int(x[1].split()[2])
        label = x[1].split()[0]
        
        offsets.append((start, 'S', label, entity_id))
        offsets.append((end, 'E', label, entity_id))
    
    # sort offsets and clean overlapping entries
    sorted_offsets = sorted(offsets, key=lambda x:x[0])
    
    cleaned_offsets = []
    corrections = {}
    
    hold = None
    indicator = None
    
    for tup in sorted_offsets:
        
        if indicator == 'S':
            if tup[1] == 'E':
                cleaned_offsets.append(hold)
                hold = (tup[0], 'O', 'X')
                indicator = tup[1]
            elif tup[1] == 'S':
                corrections.update({tup[3]:hold[2]})
                indicator = '*'
        
        elif indicator == 'E':
            cleaned_offsets.append(hold)
            hold = (tup[0], tup[2], tup[3])
            indicator = tup[1]
        
        elif indicator == '*':
            indicator = 'S'

        else:
            hold = (tup[0], tup[2], tup[3])
            indicator = tup[1]
            
    cleaned_offsets.append(hold)
    
    # extract information for identifying relations
    relations = []
    
    for r in rel:
        relation_id = r[0]
        relation_type = r[1].split()[0]
        entity1 = r[1].split()[1][5:]
        entity2 = r[1].split()[2][5:]
        
        if entity1 in corrections.keys():
            entity1 = corrections[entity1]
        if entity2 in corrections.keys():
            entity2 = corrections[entity2]
        
        relations.append((relation_id, relation_type, entity1, entity2))
    
    return cleaned_offsets, relations

#### Test the Helper Functions

In [4]:
test_path = '../raw_data/sample_ee/0003'

In [5]:
with io.open(f'{test_path}.txt', 'r', encoding='utf-8', errors='ignore') as text:
    full_text = text.read()
full_text

"Example 51-5 : Preparation of 2'-amino-6-(2-amino-6-morpholinopyrimidin-4-yl)-3'-fluoro-[2,4'-bipyridin]-5-ol (LXXVI)\n6-(2-Amino-6-morpholinopyrimidin-4-yl)-3'-fluoro-5-methoxy-[2,4'-bipyridin]-2'-amine (120 mg, 301.95 µmol) and pyridine hydrochloride (Pyridine HCl) (523.41 mg, 4.53 mmol) were stirred in a sealed tube at 170 °C for 30 min. The resulting mixture was cooled to room temperature, neutralized with 2 N NaOH solution to provide a solid. The solid was filtered, washed with diethylether and dried to give the title compound (72 mg, 62 %)."

In [6]:
with io.open(f'{test_path}.ann', 'r', encoding='utf-8', errors='ignore') as text:
    ann = [x.strip().split('\t') for x in text.readlines()] #if x.strip().split('\t')[0][0] == 'R']
ann[:3]

[['T0', 'TIME 334 340', '30 min'],
 ['T1', 'TEMPERATURE 378 394', 'room temperature'],
 ['T2', 'YIELD_PERCENT 545 549', '62 %']]

In [7]:
test_offsets, test_relations = process_ann(f'{test_path}.ann')

In [8]:
test_offsets[:3]

[(8, 'EXAMPLE_LABEL', 'T9'), (12, 'O', 'X'), (30, 'REACTION_PRODUCT', 'T7')]

In [9]:
test_relations[:3]

[('R0', 'ARG1', 'T14', 'T13'),
 ('R1', 'ARG1', 'T15', 'T12'),
 ('R2', 'ARGM', 'T15', 'T6')]

#### EDA on training data

In [10]:
path_train = '../raw_data/EE/ee_train'
filenames_train = list({x[:4] for x in os.listdir(path_train) if x[0] != '.'})
print(f'Number of train files: {len(filenames_train)}')

Number of train files: 900


In [11]:
positive_relations = []
unique_triggers = []
entities = []
example_labels = []
example_check = []
pairs_per_trigger = []
before_count = []
span_before = []
after_count = []
span_after = []

for file in filenames_train:
    
    offsets, relations = process_ann(f'{path_train}/{file}.ann')
    
    # how many positive relations per snippet?
    positive_relations.append(len(relations))
    
    # how many unique trigger words per snippet?
    arg1 = {x[2] for x in relations}
    unique_triggers.append(len(arg1))
    
    arg2 = {x[3] for x in relations}
    example = []
    for x in offsets:
        if x[1] == 'EXAMPLE_LABEL':
            example.append(x[2])
    
    # how many example labels per snippet?
    example_labels.append(len(example))
    
    # is example label ever in a relation?
    for x in example:
        if x in arg2:
            example_check.append(1)
        else:
            example_check.append(0)
    
    # how many pairs per trigger word?
    for x in arg1:
        count = [1 for r in relations if r[2] == x]
        pairs_per_trigger.append(len(count))
        
    # what is the span?
    entity_order = [x[2] for x in offsets if (x[2] != 'X' and x[1] != 'EXAMPLE_LABEL')]
    entities.append(len(entity_order))
    
    for x in arg1:
        index = entity_order.index(x)
        rels = [entity_order.index(r[3]) for r in relations if r[2] == x]
        
        bf = [x for x in rels if x < index]
        if bf:
            before_count.append(len(bf))
            span = index - min(bf)
            span_before.append(span)
        
        af = [x for x in rels if x > index]
        if af:
            after_count.append(len(af))
            span = max(af) - index
            span_after.append(span)

In [12]:
print(f'Is example label ever in a relation? {set(example_check)}')
print(f'Max example labels in train set: {max(example_labels)}')

Is example label ever in a relation? {0}
Max example labels in train set: 2


In [13]:
print(f'Total positive relations in train set: {sum(positive_relations)}')
print()
print(f'Total unique trigger words in train set: {sum(unique_triggers)}')
print(f'Total number of entities in train set (excluding EXAMPLE_LABEL): {sum(entities)}')
print()
print(f'Median unique trigger words in train set: {median(unique_triggers):.1f}')
print(f'Average unique trigger words in train set: {mean(unique_triggers):.1f}')
print(f'Max unique trigger words in train set: {max(unique_triggers)}')
print()
print(f'Median pairs per trigger word in train set: {median(pairs_per_trigger):.1f}')
print(f'Average pairs per trigger word in train set: {mean(pairs_per_trigger):.1f}')
print(f'Max pairs per trigger word in train set: {max(pairs_per_trigger)}')
print()
print(f'Median entities in train set: {median(entities):.1f}')
print(f'Average entities in train set: {mean(entities):.1f}')
print(f'Max entities in train set: {max(entities)}')

Total positive relations in train set: 14310

Total unique trigger words in train set: 6852
Total number of entities in train set (excluding EXAMPLE_LABEL): 22297

Median unique trigger words in train set: 7.0
Average unique trigger words in train set: 7.6
Max unique trigger words in train set: 62

Median pairs per trigger word in train set: 2.0
Average pairs per trigger word in train set: 2.1
Max pairs per trigger word in train set: 13

Median entities in train set: 24.0
Average entities in train set: 24.8
Max entities in train set: 182


In [14]:
print('# of pairs   count')

for i in range(max(pairs_per_trigger)):
    count = [1 for x in pairs_per_trigger if x == (i+1)]
    print(f'{(i+1):^10}   {len(count)}')

# of pairs   count
    1        2985
    2        1674
    3        1395
    4        410
    5        253
    6        92
    7        24
    8        14
    9        1
    10       2
    11       0
    12       0
    13       2


In [15]:
print(f'Median entities before trigger word in train set: {median(before_count):.1f}')
print(f'Average entities before trigger word in train set: {mean(before_count):.1f}')
print(f'Max entities before trigger word in train set: {max(before_count)}')
print()
print(f'Median entities after trigger word in train set: {median(after_count):.1f}')
print(f'Average entities after trigger word in train set: {mean(after_count):.1f}')
print(f'Max entities after trigger word in train set: {max(after_count)}')

Median entities before trigger word in train set: 2.0
Average entities before trigger word in train set: 1.8
Max entities before trigger word in train set: 8

Median entities after trigger word in train set: 2.0
Average entities after trigger word in train set: 1.8
Max entities after trigger word in train set: 13


In [16]:
more_than_before = [x for x in before_count if x > 5]
more_than_after = [x for x in after_count if x > 5]

print(f'Number of trigger words with more than 5 linked entities before: {len(more_than_before)}')
print(f'Number of trigger words with more than 5 linked entities after: {len(more_than_after)}')

Number of trigger words with more than 5 linked entities before: 17
Number of trigger words with more than 5 linked entities after: 16


In [17]:
print(f'Median span before trigger word in train set: {median(span_before):.1f}')
print(f'Average span before trigger word in train set: {mean(span_before):.1f}')
print(f'Max span before trigger word in train set: {max(span_before)}')
print()
print(f'Median span after trigger word in train set: {median(span_after):.1f}')
print(f'Average span after trigger word in train set: {mean(span_after):.1f}')
print(f'Max span after trigger word in train set: {max(span_after)}')

Median span before trigger word in train set: 2.0
Average span before trigger word in train set: 1.9
Max span before trigger word in train set: 8

Median span after trigger word in train set: 2.0
Average span after trigger word in train set: 1.8
Max span after trigger word in train set: 13


In [18]:
span_more_than_before = [x for x in span_before if x > 5]
span_more_than_after = [x for x in span_after if x > 5]

print(f'Number of trigger words with span more than 5 linked entities before: {len(span_more_than_before)}')
print(f'Number of trigger words with span more than 5 linked entities after: {len(span_more_than_after)}')

Number of trigger words with span more than 5 linked entities before: 18
Number of trigger words with span more than 5 linked entities after: 26


In [19]:
before_leftout = [x-5 for x in span_more_than_before]
after_leftout = [x-5 for x in span_more_than_after]

total_leftout = sum(before_leftout) + sum(after_leftout)
fraction_leftout = total_leftout / sum(positive_relations)

print(f'Number of relations left out: {total_leftout}')
print(f'Fraction left out: {fraction_leftout:.3f}')

Number of relations left out: 77
Fraction left out: 0.005


#### EDA revisted with updated processing function

In [20]:
def process_ann2(ann_file):
    """Helper function that reads a .ann file,
       strips out newline characters, splits the tab-delimited entries,
       and extracts information for identifying entities and relations 
       in corresponding .txt file.
       Also adds negative relations within the span of +/- entities 
       from each trigger word.
       
       Input:
       ann_file = tab-delimited brat annotation file with the following format
                  NER: [entity_ID]\t[label start_offset end_offset]\t[entity]
                  RE:  [relation_ID]\t[relation_type argument1 argument2]
       
       Outputs:
       cleaned_offsets = list of tuples for labeling corresponding .txt file
                         format: (offset, label, entity ID)
       relations = list of tuples for extracting relations from corresponding .txt file
                   format: (relation ID, relation_type, entity ID #1, entity ID #2)"""
    
    with io.open(ann_file, 'r', encoding='utf-8', errors='ignore') as f:
        text = [x.strip().split('\t') for x in f.readlines()]
        
    ann = [x for x in text if x[0][0] == 'T']
    rel = [x for x in text if x[0][0] == 'R']
    
    # extract information for identifying entities
    offsets = []
    
    for x in ann:
        entity_id = x[0]
        start = int(x[1].split()[1])
        end = int(x[1].split()[2])
        label = x[1].split()[0]
        
        offsets.append((start, 'S', label, entity_id))
        offsets.append((end, 'E', label, entity_id))
    
    # sort offsets and clean overlapping entries
    sorted_offsets = sorted(offsets, key=lambda x:x[0])
    
    cleaned_offsets = []
    corrections = {}
    
    hold = None
    indicator = None
    
    for tup in sorted_offsets:
        
        if indicator == 'S':
            if tup[1] == 'E':
                cleaned_offsets.append(hold)
                hold = (tup[0], 'O', 'X')
                indicator = tup[1]
            elif tup[1] == 'S':
                corrections.update({tup[3]:hold[2]})
                indicator = '*'
        
        elif indicator == 'E':
            cleaned_offsets.append(hold)
            hold = (tup[0], tup[2], tup[3])
            indicator = tup[1]
        
        elif indicator == '*':
            indicator = 'S'

        else:
            hold = (tup[0], tup[2], tup[3])
            indicator = tup[1]
            
    cleaned_offsets.append(hold)
    
    # extract information for identifying relations
    relations = []
    positives = []
    
    # add positive relations
    for r in rel:
        relation_id = r[0]
        relation_type = r[1].split()[0]
        entity1 = r[1].split()[1][5:]
        entity2 = r[1].split()[2][5:]
        
        if entity1 in corrections.keys():
            entity1 = corrections[entity1]
        if entity2 in corrections.keys():
            entity2 = corrections[entity2]
        
        relations.append((relation_id, relation_type, entity1, entity2))
        positives.append((entity1, entity2))
    
    # negative relations
    negatives = []
    triggers = {x[2] for x in cleaned_offsets if (x[1] == 'WORKUP' or x[1] == 'REACTION_STEP')}
    entity_order = [x[2] for x in cleaned_offsets if (x[2] != 'X' and x[1] != 'EXAMPLE_LABEL')]
    
    # find negative relations
    for t in triggers:
        index = entity_order.index(t)
        
        # find indices in span of +/- 5 from trigger
        find_span = [index - (i+1) for i in range(5)] + [index + (i+1) for i in range(5)]
        span = [i for i in find_span if (i >= 0 and i < len(entity_order))]
        
        # make tuples of trigger words and entities from span indices
        potential_pairs = [(t, entity_order[i]) for i in span]
        
        # check if tuple is in positives
        # only add to negatives if not in positives
        for pair in potential_pairs:
            if pair in positives:
                continue
            else:
                negatives.append(pair)
    
    # make a list of negative relations 
    # in same format as positives to add to relations list
    i = 0
    for pair in negatives:
        negative_id = f'N{i}'
        relations.append((negative_id, 'NONE', pair[0], pair[1]))
        i += 1
    
    return cleaned_offsets, relations

In [21]:
negative_relations = []

for file in filenames_train:
    
    offsets, relations = process_ann2(f'{path_train}/{file}.ann')
    negatives = [1 for x in relations if x[0][0] == 'N']
    negative_relations.append(len(negatives))

In [22]:
print(f'Total negative relations in train set: {sum(negative_relations)}')
print()
print(f'Median negative relations in train set: {median(negative_relations):.1f}')
print(f'Average negative relations in train set: {mean(negative_relations):.1f}')
print(f'Max negative relations in train set: {max(negative_relations)}')

Total negative relations in train set: 50685

Median negative relations in train set: 53.0
Average negative relations in train set: 56.3
Max negative relations in train set: 492
